## 設定攝影機編號
### 請使用者實際測試並設定 VIDEO_INDEX 編號值。
### 解析度(寬為640，高為480)

In [1]:
VIDEO_INDEX = 1
#VIDEO_INDEX = 'face.mp4'
#VIDEO_INDEX = 'http://192.168.0.14:5000/live'

VIDEO_WIDTH = 640
VIDEO_HEIGHT = 480

## 匯入相依套件

In [2]:
import cv2
import numpy as np
import math
import mediapipe as mp
import matplotlib.pyplot as plt
from IPython.display import display, Image
import ipywidgets as widgets
import threading

## 設定mediapipe功能為臉部特徵追蹤功能

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

## 設定按鈕的外型

In [4]:
stopButton = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='square' # (FontAwesome names without the `fa-` prefix)
)

In [5]:
Display_Value_Button = widgets.ToggleButton(
    value=False,
    description='Display Value',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='square' # (FontAwesome names without the `fa-` prefix)
)

## 副程式
### 顯示狀態訊息文字於指定的位置
### ratio_x 和 ratio_y 都是採用相對比例位置

In [6]:
def Display_Info(image, info, ratio_x,ratio_y):
    image_width=image.shape[1]
    image_height=image.shape[0]
    pos_x = int(image_width * ratio_x)
    pos_y = int(image_height * ratio_y)
    cv2.putText(
        image,
        info,
        (pos_x,pos_y),
        cv2.FONT_HERSHEY_SIMPLEX, 1,
        (0,255,0), 6, cv2.LINE_AA)
    cv2.putText(
        image,
        info,
        (pos_x,pos_y),
        cv2.FONT_HERSHEY_SIMPLEX, 1,
        (0,0,0), 2, cv2.LINE_AA)

## 副程式
### 計算 Point_1 和 Point_2 兩點之間的距離
### 距離是採用與螢幕比例大小進行回傳

In [7]:
def Distance_Ratio_2_Point(face_landmarks,Point_1,Point_2):
    Point_1_X = face_landmarks.landmark[Point_1].x
    Point_1_Y = face_landmarks.landmark[Point_1].y
    Point_1 = np.array([Point_1_X,Point_1_Y])

    Point_2_X = face_landmarks.landmark[Point_2].x
    Point_2_Y = face_landmarks.landmark[Point_2].y
    Point_2 = np.array([Point_2_X,Point_2_Y])
    
    Diff = Point_1 - Point_2
    Distance = math.hypot(Diff[0],Diff[1])
    
    return Distance

## OpenCV顯示即時影像畫面副程式

In [8]:
def view(button):
    
    cap = cv2.VideoCapture(VIDEO_INDEX)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, VIDEO_WIDTH)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, VIDEO_HEIGHT)
    
    sw = 1
    display_handle=display(None, display_id=True)
    
    with mp_face_mesh.FaceMesh(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as face_mesh:       
        
        stop_flag = 0
        frame_counter = 0
    
        while (stop_flag == 0):
            
            if (type(VIDEO_INDEX) == int) or ('http' in VIDEO_INDEX):
                ret,image = cap.read()
            else:
                ret,image = cap.read()
                frame_counter += 1
                if frame_counter == int(cap.get(cv2.CAP_PROP_FRAME_COUNT)):
                    frame_counter = 0
                    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)                
            if (ret != True):
                break

            # image = cv2.flip(image, 1) # if your camera reverses your image
        
            image_width=image.shape[1]
            image_height=image.shape[0]
            
            # Flip the image horizontally for a later selfie-view display, and convert the BGR image to RGB.
            image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
            # To improve performance, optionally mark the image as not writeable to pass by reference.
            image.flags.writeable = False
            results = face_mesh.process(image)

            # Draw the face mesh annotations on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:

                    DLV = Distance_Ratio_2_Point(face_landmarks,145,159) #左眼上下眼瞼垂直距離

                    DLH = Distance_Ratio_2_Point(face_landmarks,133,33) #左眼內外眼角水平距離
                                
                    DRV = Distance_Ratio_2_Point(face_landmarks,374,386) #右眼上下眼瞼垂直距離

                    DRH = Distance_Ratio_2_Point(face_landmarks,362,263) #右眼內外眼角水平距離
                    
                    
                    DRN = Distance_Ratio_2_Point(face_landmarks,374,6) #鼻子到右眼的距離
                    
                    DLN = Distance_Ratio_2_Point(face_landmarks,145,6) ##鼻子到左眼的距離

                    Ratio_L = DLV / DLH #右眼張開比例 
                    Ratio_R = DRV / DRH #左眼張開比例
                    
                    if (Ratio_L < 0.35) and (Ratio_R < 0.35) and (sw == 0):
                        Display_Info(
                            image, 
                            'Close', 
                            0.5, 0.2)
                    elif (sw == 0):
                        Display_Info(
                            image, 
                            'Normal', 
                            0.5, 0.2)
        
                    if (sw == 1):
                        mp_drawing.draw_landmarks(
                            image=image,
                            landmark_list=face_landmarks,
                            connections=mp_face_mesh.FACEMESH_CONTOURS,
                            landmark_drawing_spec=drawing_spec,
                            connection_drawing_spec=drawing_spec)
                    
                        #Display_Info(image, 'Ratio_L='+ str(Ratio_L), 0.05,0.05)
                        #Display_Info(image, 'Ratio_R='+ str(Ratio_R), 0.05,0.15)

                        Display_Info(image, 'DRN='+ str(DRN), 0.05,0.05)
                        Display_Info(image, 'DLN='+ str(DLN), 0.05,0.15)

                        
                        
                    _, image = cv2.imencode('.jpeg', image)
        
                    display_handle.update(Image(data=image.tobytes()))
            
                    if Display_Value_Button.value==True:
                        sw = sw + 1
                        if (sw == 2):
                            sw = 0
                        Display_Value_Button.value = False
            
                    if stopButton.value==True:
                        stop_flag = 1
                        cap.release()
                        display_handle.update(None)

## 執行程式

In [9]:
display(Display_Value_Button)

thread = threading.Thread(target=view, args=(stopButton,))
thread.start()

display(stopButton)

ToggleButton(value=False, button_style='info', description='Display Value', icon='square', tooltip='Descriptio…

ToggleButton(value=False, button_style='danger', description='Stop', icon='square', tooltip='Description')

None